In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, roc_auc_score, f1_score, recall_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import os

random_state = 36021

In [2]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from fancyimpute import IterativeImputer


Using TensorFlow backend.
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users

In [3]:
data_path_2009 = r'C:\Users\nogag\Documents\birocracy\PTSDClassifier\PTSD\Data\2009'
df_2009 = pd.read_excel(os.path.join(data_path_2009, "PTSD.xlsx"))


In [4]:
data_path_2016 = r'C:\Users\nogag\Documents\birocracy\PTSDClassifier\PTSD\Data\2016'
df_2016 = pd.read_csv(os.path.join(data_path_2016, "IDF_ABM_16.2.15_wide.csv"))


df_2016 = df_2016[~(df_2016['Wave']=='nov12')]


## features in the original data

In [5]:
trans_2016_2009_features = {
    'bagrut': 'highschool_diploma',
 'ADHD': 'ADHD',
 'Accuracy_threat_T1': 'T1Acc1t',
 'Accuracy_NT_T1': 'T1Acc1n',
 'Threat_Bias_T1': 'T1bias',
 'PHQ_T1': 'phq1',
 'Trait_T1': 'trait1',
 'State_T1': 'state1',
  'PCL_T1': 'PCL1',
 'PCL_T4': 'PCL3'}


## append PCL intrusion features

In [6]:
PCL_2009 = pd.read_csv(os.path.join(data_path_2009, "questionnaire_PCL1.csv"))
PCL_2016 = pd.read_csv(os.path.join(data_path_2016, "questionnaire5_PCL.csv"))                      

intrusion_features_2009 = ["q6.1_INTRU", "q6.2_DREAM", "q6.3_FLASH", "q6.4_UPSET", "q6.5_PHYS"]
intrusion_features_2016 = ["q5.1", "q5.2", "q5.3", "q5.4", "q5.5"]
df_2009 = df_2009.merge(PCL_2009[intrusion_features_2009 + ["ID"]], on="ID", how='outer')
df_2016 = df_2016.merge(PCL_2016[intrusion_features_2016 + ["ID"]], on="ID", how='outer')

for i, j in zip(intrusion_features_2009, intrusion_features_2016):
    trans_2016_2009_features[j] = i

## append PHQ9 features

In [7]:
PHQ9_2009 = pd.read_csv(os.path.join(data_path_2009, "questionnaire5_PHQ9.csv"))
PHQ9_2016 = pd.read_csv(os.path.join(data_path_2016, "questionnaire4_PHQ9.csv"))                      

PHQ9_features_2009 = ["T1q5.1", "T1q5.2", "T1q5.3", "T1q5.4", "T1q5.5", "T1q5.6", "T1q5.7", "T1q5.8", "T1q5.9"]
PHQ9_features_2016 = ["q4.1", "q4.2","q4.3", "q4.4", "q4.5", "q4.6", "q4.7", "q4.8", "q49"]
df_2009 = df_2009.merge(PHQ9_2009[PHQ9_features_2009 + ["ID"]], on="ID", how='outer')
df_2016 = df_2016.merge(PHQ9_2016[PHQ9_features_2016 + ["ID"]], on="ID", how='outer')

for i, j in zip(PHQ9_features_2009, PHQ9_features_2016):
    trans_2016_2009_features[j] = i

## append trait featues

In [8]:
Trait_2009 = pd.read_csv(os.path.join(data_path_2009, "questionnaire3_trait_anxiety_revised.csv"))
Trait_2016 = pd.read_csv(os.path.join(data_path_2016, "questionnaire3_trait_anxiety.csv"))                      

Trait_features_2009 = ["T1q3.1", "T1q3.2", "T1q3.3", "T1q3.4", "T1q3.5", "T1q3.6", "T1q3.7", "T1q3.8", "T1q3.9", "T1q3.10", "T1q3.11", "T1q3.12", "T1q3.13", "T1q3.14", "T1q3.15", "T1q3.16", "T1q3.17", "T1q3.18", "T1q3.19", "T1q3.20"]
Trait_features_2016 = ["q3.1", "q3.2", "q3.3", "q3.4", "q3.5", "q3.6", "q3.7", "q3.8", "q3.9", "q3.10", "q3.11", "q3.12", "q3.13", "q3.14", "q3.15", "q3.16", "q3.17", "q3.18", "q3.19", "q3.20"]
df_2009 = df_2009.merge(Trait_2009[Trait_features_2009 + ["ID"]], on="ID", how='outer')
df_2016 = df_2016.merge(Trait_2016[Trait_features_2016 + ["ID"]], on="ID", how='outer')

for i, j in zip(Trait_features_2009, Trait_features_2016):
    trans_2016_2009_features[j] = i

## append state featues

In [9]:
State_2009 = pd.read_csv(os.path.join(data_path_2009, "questionnaire2_state_anxiety_revised.csv"))
State_2016 = pd.read_csv(os.path.join(data_path_2016, "questionnaire2_state_anxiety.csv"))                      

State_features_2009 = ["T1q2.1", "T1q2.2", "T1q2.3", "T1q2.4", "T1q2.5", "T1q2.6", "T1q2.7", "T1q2.8", "T1q2.9", "T1q2.10", "T1q2.11", "T1q2.12", "T1q2.13", "T1q2.14", "T1q2.15", "T1q2.16", "T1q2.17", "T1q2.18", "T1q2.19", "T1q2.20"]
State_features_2016 = ["q2.1", "q2.2", "q2.3", "q2.4", "q2.5", "q2.6", "q2.7", "q2.8", "q2.9", "q2.10", "q2.11", "q2.12", "q2.13", "q2.14", "q2.15", "q2.16", "q2.17", "q2.18", "q2.19", "q2.20"]
df_2009 = df_2009.merge(State_2009[State_features_2009 + ["ID"]], on="ID", how='outer')
df_2016 = df_2016.merge(State_2016[State_features_2016 + ["ID"]], on="ID", how='outer')

for i, j in zip(State_features_2009, State_features_2016):
    trans_2016_2009_features[j] = i

## target feature

In [10]:
target_feature = 'PCL3'
X_features = [i for i in trans_2016_2009_features.values() if not i == target_feature]


In [11]:
df_2016['PCL_T4'] = (df_2016['PCL_T4'] > 40).astype(int)
df_2009['PCL3'] = (df_2009['PCL3'] > 40).astype(int)

## adjust features from 2016

In [12]:
df_2016['bagrut'] = df_2016['bagrut'] == 'yes'
df_2016['dyslexia'] = df_2016['dyslexia'] == 'yes'
df_2016['ADHD'] = df_2016['ADHD'] == 'yes'

In [13]:
df_2016 = df_2016.rename(trans_2016_2009_features, axis=1)

In [14]:
df_2009 = df_2009[~df_2009[target_feature].isna()]
df_2016 = df_2016[~df_2016[target_feature].isna()]

## 2009 data outer CV

In [15]:
x, x_test, y, y_test = train_test_split(df_2009[X_features], df_2009[target_feature], test_size=0.25,
                                        random_state=random_state, stratify=df_2009[target_feature])

## parameters init

In [29]:
? IterativeImputer

In [36]:
cv = StratifiedKFold(6, random_state=random_state, shuffle=True)

# class weight
pos_sample = y.sum() 
all_samples = y.count()
class_weights = all_samples/ pos_sample


# pipeline

pipe = Pipeline(steps=[
    ('Mice',  IterativeImputer(random_state=random_state, max_iter=65)),
    ('RFE', RFE(ExtraTreesClassifier(random_state=random_state))),
    ('classifier', CatBoostClassifier(verbose=0, random_state=random_state))

    ])
grid_params = [{
        'RFE__n_features_to_select': [15, 35, 55],
        #'classifier__bagging_temperature': [1, 10, 50],#, 9]
        #'classifier__learning_rate':[None, 0.01, 0.1],
        'classifier__class_weights':[[1, class_weights]],# [1, class_weights*2]],# [1, class_weights*0.5]],
        'classifier__l2_leaf_reg': [150, 3],# 250, 500],
        #'classifier__depth': [4, 7]#, 9]
        }]




## inner CV

In [37]:
x_train, x_val, y_train, y_val = train_test_split(x, y,  random_state=random_state, test_size=0.2, stratify=y)
## grid search
clf = GridSearchCV(pipe, grid_params, cv=cv, scoring='roc_auc')

In [38]:
clf.fit(x_train, y_train.values.astype(int), classifier__early_stopping_rounds = 15)
print(f"roc_auc = {clf.best_score_}, params = {clf.best_params_}")


roc_auc = 0.6735252915441617, params = {'RFE__n_features_to_select': 55, 'classifier__class_weights': [1, 13.573770491803279], 'classifier__l2_leaf_reg': 150}


In [39]:
y_pred_target = clf.best_estimator_.predict_proba(x_val)[:, 1]
print(f"roc_auc = {roc_auc_score(y_val.astype(int), y_pred_target)}")

roc_auc = 0.799512987012987


## outer CV

In [40]:
clf = GridSearchCV(pipe, grid_params, cv=cv, scoring='roc_auc')
clf.fit(x, y.values.astype(int), classifier__early_stopping_rounds = 15)

print(f"roc_auc = {clf.best_score_}, params = {clf.best_params_}")

roc_auc = 0.6703796080887616, params = {'RFE__n_features_to_select': 55, 'classifier__class_weights': [1, 13.573770491803279], 'classifier__l2_leaf_reg': 3}


In [41]:
y_pred_target = clf.best_estimator_.predict_proba(x_test)[:, 1]
print(f"roc_auc = {roc_auc_score(y_test.astype(int), y_pred_target)}")

roc_auc = 0.7109922178988327


## 2016

In [42]:
x_2016, y_2016 = df_2016[X_features], df_2016[target_feature]
clf = GridSearchCV(pipe, grid_params, cv=cv, scoring='roc_auc')


In [43]:
clf.fit(df_2009[X_features], df_2009[target_feature].astype(int), classifier__early_stopping_rounds = 15)
print(f"roc_auc = {clf.best_score_}, params = {clf.best_params_}")

C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:686: ConvergenceWarning: [IterativeImputer] Early st

roc_auc = 0.6780405639538766, params = {'RFE__n_features_to_select': 35, 'classifier__class_weights': [1, 13.573770491803279], 'classifier__l2_leaf_reg': 150}


In [44]:
y_pred_target = clf.best_estimator_.predict_proba(x_2016)[:, 1]
print(f"roc_auc = {roc_auc_score(y_2016.astype(int), y_pred_target)}")

roc_auc = 0.4901079895385134


In [45]:
for i, j in zip(x_train.columns, clf.best_estimator_['classifier'].get_feature_importance()):
    print(i, j)

highschool_diploma 3.861605010719701
ADHD 3.3598636410587335
T1Acc1t 3.3481488204795333
T1Acc1n 6.519837822341892
T1bias 4.448840522602901
phq1 1.9227397722132664
trait1 3.062935294603366
state1 9.731166664140055
PCL1 1.6126299121046557
q6.1_INTRU 2.1672486008953196
q6.2_DREAM 1.9111880015127454
q6.3_FLASH 1.8534408834285447
q6.4_UPSET 1.4828224732678412
q6.5_PHYS 2.9729834830131816
T1q5.1 3.4354634386258605
T1q5.2 3.6613550985585785
T1q5.3 1.889573812313855
T1q5.4 3.7427144356320756
T1q5.5 3.4560774106603107
T1q5.6 1.7260383621898596
T1q5.7 2.8537370706839162
T1q5.8 1.5123887633563853
T1q5.9 1.900691651501232
T1q3.1 2.800906408583409
T1q3.2 2.0892137766339425
T1q3.3 4.0404428289153635
T1q3.4 3.367204329168988
T1q3.5 1.8209417802936976
T1q3.6 3.013473002746697
T1q3.7 1.875279508638163
T1q3.8 1.7375756499194293
T1q3.9 0.8248161966384734
T1q3.10 1.2730630332702617
T1q3.11 3.2620430248245897
T1q3.12 1.4615495144632098


In [46]:
y_2016.astype(int)

0       0
1       0
2       0
3       0
4       0
       ..
1348    0
1349    0
1350    0
1351    0
1352    0
Name: PCL3, Length: 1353, dtype: int32

In [47]:
roc_auc_score(y_2016.astype(int), y_pred_target)

0.4901079895385134